# Readme

Instructions for implementing following code:

1. Download and Install Anaconda distribution from https://www.anaconda.com/products/individual. 
User guide can be found at https://docs.anaconda.com/anaconda/user-guide/getting-started/.
2. Download and Install Julia from https://julialang.org/downloads/
3. Open julia, and enter 'import Pkg', followed by 'Pkg.add("IJulia")'
4. In the Pkg environment, enter the following 'add("IJulia")','add("JuMP")','add("Cbc")','add("Ipopt")'
5. Open Jupyter notebook from Anaconda Navigator, and open this file. 
6. Implement the code by pressing shift+enter in each cell.

Content:

1. Case 2 Analysis and Synthesis problem solution
2. Case 3 Analysis and Synthesis problem solution
3. Illustrative Example solution for multiple objectives
4. Appendix A.1. Solutions


In [5]:
using JuMP, Gurobi

# Case 2: Downstream Alternatives

## Analysis:

In [6]:
A=[[2 -4 -2 0];[-1 1 0.45 -1];[0 4 2 0]]
w1=0.5
A_allocation=[[2*w1 -4 0 0];[2*(1-w1) 0 -2 0];[-1 1 0.45 -1];[0 4 2 0]]
f=[0;0;0;100]
s=inv(A_allocation)*f
B=[[1 1 1 0];[-3 0 0 0]]
g=B*s
g

2-element Vector{Float64}:
   87.5
 -150.0

## Synthesis:

In [7]:
model = Model(with_optimizer(Gurobi.Optimizer))

A=[[2 -4 -2];[-1 1 0.45]]
B=[[1 1 1];[-3 0 0]]
@variables(model, begin
    f[1:2]
    g[1:2] 
    s[1:3] ≥ 0
end)
@constraints(model, begin
    A*s .== f #LC,A
    f[1] == 0
    #2*s[1]-4*s[2]-2*s[3]==0
    #-1*s[1]+s[2]+0.45*s[3]==f[2]
    #s[1]==50
    B*s .== g #LC
    4*s[2]+2*s[3] == 100 #U
end)

@objective(model, Min, g[1])
JuMP.optimize!(model)
println("The solution is $(termination_status(model))")

println("The model is :")
print(model)
println("\n\nTotal Flow (Objective value): ", JuMP.objective_value(model))
println("\nScaling Factors:")
print(JuMP.value.(s))
println("\nFinal Demands:")
print(JuMP.value.(f))

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-07
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 6 rows, 7 columns and 17 nonzeros
Model fingerprint: 0x67d6f315
Coefficient statistics:
  Matrix range     [5e-01, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 6 rows and 7 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds (0.00 work units)
Optimal objective  7.500000000e+01

User-callback calls 36, time in user-callback 0.00 sec
The solution is OPTIMAL
The model is :


Total Flow (Objective value): 75.0

Scaling Factors:
[50.0, 25.0, 0.0]
Final Demands:
[0.0, -25.0]

# Case 3: Upstream Alternatives

## Analysis:


In [8]:
A=[[-1.5 2.5];[-0.5 0.5];[2 -4];[2 0]]
B=[[1 1];[0 0.5];[0 0.5]]
A_CE=[[-1.5 2.5 -1 0];[-0.5 0.5 0 -1];[2 -4 0 0];[2 0 0 0]]
B_CE=[[1 1 0 0];[0 0.5 0 0];[0 0.5 0 0]]
f=[0;0;0;100]
s=inv(A_CE)*f
g=B_CE*s
g

3-element Vector{Float64}:
 75.0
 12.5
 12.5

## Synthesis:


In [11]:
model = Model(with_optimizer(Gurobi.Optimizer,NonConvex=2))

@variables(model, begin
    f[1:3]
    g[1:3] 
    s[1:2] ≥ 0
    a_var[1:2,1:2]
    b_5[1:2] ≥ 0
    x[1:2] ≥ 0
end)
@constraints(model, begin
    #A*s.==f #LC,A
    #B*s.==g #LC
    a_var[1:2,1].≤ 0
    a_var[1:2,2].≥ 0
    constr1[i=1:2], sum(a_var[i,j]*s[j] for j in 1:2) == f[i]
    constr2[i=2:3], b_5[i-1]*s[2] == g[i]
    sum(A[3,j]*s[j] for j in 1:2) == f[3]
    f[3]==0
    
    sum(B[1,j]*s[j] for j in 1:2) == g[1]
    sum(a_var[i,1] for i in 1:2)==-1*A[3,1]
    x[1]*(-2)==a_var[1,1]
    x[2]*(-2)==a_var[2,1]
    2*s[1]*x[1]==a_var[1,2]*s[2]+b_5[1]*s[2]
    2*s[1]*x[2]==a_var[2,2]*s[2]+b_5[2]*s[2]
    a_var[1,2]==b_5[1]
    a_var[2,2]==2*b_5[2]
    -1*(sum(a_var[i,1] for i in 1:2))*s[1]==100 #U
end)

@objective(model, Min, g[1])
JuMP.optimize!(model)
println("The solution is $(termination_status(model))")

println("\n\nTotal Flow (Objective value): ", JuMP.objective_value(model))
println("\nScaling Factors:")
print(JuMP.value.(s))
println("\nFinal Demands:")
print(JuMP.value.(f))

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-07
Set parameter NonConvex to value 2
Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 12 rows, 16 columns and 21 nonzeros
Model fingerprint: 0xa6c02e4d
Model has 7 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+00]
  QRHS range       [1e+02, 1e+02]
Presolve removed 6 rows and 2 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 11 rows and 7 columns
Presolve time: 0.01s
Presolved: 30 rows, 14 columns, 64 nonzeros
Presolved model has 6 bilinear constraint(s)
Variable types: 14 continuous, 0 integer (0 binary)

Root relaxation: objective 

# Section 4.2 Illustrative Example:

## With All Alternatives and Multiple-Objectives

In [13]:
model = Model(with_optimizer(Gurobi.Optimizer,NonConvex=2))
#   RM E P1 P2 C R1 R2 SW DC  
A=[ [1 0 -20 -40 0 30 45 0 0];
    [0 1 -1 -2 0 0 0 0 0.1];
    [0 0 100 0 -1 0 0 0 0];
    [0 0 0 100 -1 0 0 0 0];
    [0 0 0 0 1 0 0 -30 -20];
    [0 0 0 0 0 -50 -50 22 0]]

B=[[0.25 0.2 2 1 0 2 5 1 1];
    [0 0 0 0 0.4 5 5 0 2];
    [0 0 0 0 0 15 0 8 5]]

m=[0.2 0.4]
@variables(model, begin
    f[1:6] ≥ 0
    g[1:3] ≥ 0
    s[1:9] ≥ 0
    x[1:2] ≥ 0 #variables in a_consumer
    Dc ≥ 0
    gCO2 ≥ 0
end)
@constraints(model, begin
    constr1[i=1:2], sum(A[i,j]*s[j] for j in 1:9) == f[i]
    constr2[i=3:4], sum(A[i,j]*s[j] for j in 1:9 if j!=5)+sum(A[i,j]*s[j]*x[i-2] for j in 1:9 if j==5) == f[i]
    constr3[i=5:6], sum(A[i,j]*s[j] for j in 1:9) == f[i]
    s[6]*((m[1]*x[1])+(m[2]*x[2]))==m[1]*x[1]*(s[6]+s[7])
    s[7]*((m[1]*x[1])+(m[2]*x[2]))==m[2]*x[2]*(s[6]+s[7]) # MASS-BASED ALLOCATION
    B*s.==g
    (x[1]+x[2])*s[5]==300
    ((m[1]*x[1])+(m[2]*x[2]))==1+B[2,5]
    constr[i=1:6],f[i]==0
end)

@constraint(model, Dc*(s[1]+0.2*s[2])==(0.8*A[1,6]*s[6]+0.8*A[1,7]*s[7]+A[2,9]*s[9]*0.2))
@objective(model, Min, g[1]) #Impact(GWP)
JuMP.optimize!(model)
print(model)
println("\n\nTotal Flow (Objective value): ", JuMP.objective_value(model))
println("\nScaling Factors:\n RM      Elec  P1   P2      C      R1      R2      SW     DC")
print(round.(JuMP.value.(s),digits=3))
println("\nItems chosen:\nP1    P2")
print(round.(JuMP.value.(100*s[3:4]),digits=0))
println("\nFinal Demands:")
print(JuMP.value.(f))
println("\nDc:")
print(JuMP.value(Dc))
println("\n")

@NLobjective(model, Max, Dc) #Dc
JuMP.optimize!(model)
print(model)
println("\n\nTotal Flow (Objective value): ", JuMP.objective_value(model))
println("\nScaling Factors:\n RM      Elec  P1   P2      C      R1      R2      SW     DC")
print(round.(JuMP.value.(s),digits=3))
println("\nItems chosen:\nP1    P2")
print(round.(JuMP.value.(100*s[3:4]),digits=0))
println("\nFinal Demands:")
print(JuMP.value.(f))
println("\ngCO2:")
print(JuMP.value(g[1]))
println("\n")

@objective(model, Min, (s[1] +(s[2]*0.25))) #Cs
JuMP.optimize!(model)
print(model)
println("\n\nTotal Flow (Objective value): ", JuMP.objective_value(model))
println("\nScaling Factors:\n RM      Elec  P1   P2      C      R1      R2      SW     DC")
print(round.(JuMP.value.(s),digits=3))
println("\nItems chosen:\nP1    P2")
print(round.(JuMP.value.(100*s[3:4]),digits=0))
println("\nFinal Demands:")
print(JuMP.value.(f))
println("\n")



@NLobjective(model, Max, Dc) #Trade-off solution
@constraint(model, g[1] <=25.5) #e-constraint method
JuMP.optimize!(model)
print(model)
println("\n\nTotal Flow (Objective value): ", JuMP.objective_value(model))
println("\nScaling Factors:\n RM      Elec  P1   P2      C      R1      R2      SW     DC")
print(round.(JuMP.value.(s),digits=3))
println("\nItems chosen:\nP1    P2")
print(round.(JuMP.value.(100*s[3:4]),digits=0))
println("\nFinal Demands:")
print(JuMP.value.(f))
println("\ngCO2:")
print(JuMP.value(g[1]))
println("\n")



Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-07
Set parameter NonConvex to value 2
Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 14 rows, 22 columns and 45 nonzeros
Model fingerprint: 0x9b66faeb
Model has 6 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-01, 5e+01]
  QMatrix range    [2e-01, 1e+00]
  QLMatrix range   [2e-02, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [3e+02, 3e+02]
Presolve removed 9 rows and 10 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 10 rows and 11 columns
Presolve time: 0.00s
Presolved: 39 rows, 18 columns, 88 nonzeros
Presolved model has 8 bilinear constraint(s)
Variable types: 18 continuous, 0 integer (0 binary)

Root relaxation: objectiv

LoadError: MathOptInterface.UnsupportedAttribute{MathOptInterface.NLPBlock}: Attribute MathOptInterface.NLPBlock() is not supported by the model.

## Component Balance constraint instead of Mass-Ratio based allocation constraint 

### (Figure 9, and Table 3 of Main paper)



In [4]:
model = Model(with_optimizer(Ipopt.Optimizer, print_level = 0))
#   RM E P1 P2 C R1 R2 SW DC  
A=[ [1 0 -20 -40 0 30 45 0 0];
    [0 1 -1 -2 0 0 0 0 0.1];
    [0 0 100 0 -1 0 0 0 0];
    [0 0 0 100 -1 0 0 0 0];
    [0 0 0 0 1 0 0 -30 -20];
    [0 0 0 0 0 -50 -50 22 0]]

B=[[0.25 0.2 2 1 0 2 5 1 10];
    [0 0 0 0 0.4 5 5 0 2];
    [0 0 0 0 0 15 0 8 5]]

m=[0.2 0.4]
@variables(model, begin
    f[1:6] ≥ 0
    g[1:3] ≥ 0
    s[1:9] ≥ 0
    x[1:2] ≥ 0  #a_c variable 
    xs[1:2] ≥ 0 #a_s variable
    xl[1:2] ≥ 0 #b_s variable
    -0.5 ≤ slack ≤ 0.5
end)
@constraints(model, begin
    constr1[i=1:2], sum(A[i,j]*s[j] for j in 1:9) == f[i]
    constr2[i=3:4], sum(A[i,j]*s[j] for j in 1:9 if j!=5)+sum(A[i,j]*s[j]*x[i-2] for j in 1:9 if j==5) == f[i]
    constr3[i=5:6], sum(A[i,j]*s[j] for j in 1:9) == f[i]
    xs[1]*(A[6,6]*s[6]+A[6,7]*s[7])==A[6,6]*s[6]
    xs[2]*(A[6,6]*s[6]+A[6,7]*s[7])==A[6,7]*s[7]
    (B[2,8]+B[3,8])*s[8]*xl[1]-A[6,6]*s[6]==-1*A[5,8]*s[8]*m[1]*x[1]/1.4
    (B[2,8]+B[3,8])*s[8]*xl[2]-A[6,7]*s[7]==-1*A[5,8]*s[8]*m[2]*x[2]/1.4
    xs[2]==xl[2]
    -1*A[6,7]*s[7]<=(m[2]*x[2]*s[5])+((A[5,9]+B[2,9]+B[3,9])*m[2]*x[2]*s[9]/1.4)-((B[2,8]+B[3,8])*s[8]*xl[2])
    B*s.==g
    (x[1]+x[2])*s[5]==300
    (m[1]*x[1])+(m[2]*x[2])==1+B[2,5]
    constr[i=1:6],f[i]==0
    #f[1]==0
end)
@objective(model, Min, g[1]) #Impact(GWP)
JuMP.optimize!(model)

print(model)
println("\n\nTotal Flow (Objective value): ", JuMP.objective_value(model))
println("\nScaling Factors:\n RM      Elec  P1   P2      C      R1      R2      SW     DC")
print(round.(JuMP.value.(s),digits=3))
println("\nItems chosen:\nP1    P2")
print(round.(JuMP.value.(100*s[3:4]),digits=0))
println("\nFinal Demands:")
print(JuMP.value.(f))


@NLobjective(model, Max, (0.8*A[1,6]*s[6]+0.8*A[1,7]*s[7]+A[2,9]*s[9]*0.2)/(s[1]+0.2*s[2])) #Dc
JuMP.optimize!(model)
print(model)
println("\n\nTotal Flow (Objective value): ", JuMP.objective_value(model))
println("\nScaling Factors:\n RM      Elec  P1   P2      C      R1      R2      SW     DC")
print(round.(JuMP.value.(s),digits=3))
println("\nItems chosen:\nP1    P2")
print(round.(JuMP.value.(100*s[3:4]),digits=0))
println("\nFinal Demands:")
print(JuMP.value.(f))

LoadError: UndefVarError: Ipopt not defined

## Technology Matrix and Intervention Matrix

In [13]:
view(A,1:6,1:9)

6×9 view(::Array{Float64,2}, 1:6, 1:9) with eltype Float64:
 1.0  0.0  -20.0  -40.0   0.0   30.0   45.0    0.0    0.0
 0.0  1.0   -1.0   -2.0   0.0    0.0    0.0    0.0    0.1
 0.0  0.0  100.0    0.0  -1.0    0.0    0.0    0.0    0.0
 0.0  0.0    0.0  100.0  -1.0    0.0    0.0    0.0    0.0
 0.0  0.0    0.0    0.0   1.0    0.0    0.0  -30.0  -20.0
 0.0  0.0    0.0    0.0   0.0  -50.0  -50.0   22.0    0.0

In [14]:
view(B,1:3,1:9)

3×9 view(::Array{Float64,2}, 1:3, 1:9) with eltype Float64:
 0.25  0.2  2.0  1.0  0.0   2.0  5.0  1.0  10.0
 0.0   0.0  0.0  0.0  0.4   5.0  5.0  0.0   2.0
 0.0   0.0  0.0  0.0  0.0  15.0  0.0  8.0   5.0

# Appendix A.1.2: Recycle Loop Optimization Problem

In [15]:
model = Model(with_optimizer(Cbc.Optimizer, LogLevel=1))
A=[[1 0 -20 0 30];[0 1 -1 0 0];[0 0 100 -7 0];[0 0 0 1 -50]]
B=[[0.25 0.2 2 0 2];[0 0 0 0.4 5];[0 0 0 0 15]]
@variables(model, begin
    f[1:4] ≥ 0
    g[1:3] ≥ 0
    s[1:5] ≥ 0
end)
@constraints(model, begin
    A*s.==f
    B*s.==g
    7*s[4]==300
    constr[i=3:4],f[i]==0
end)

@objective(model, Min, g[1])
JuMP.optimize!(model)

println("\n\nTotal Flow (Objective value): ", JuMP.objective_value(model))
println("\nScaling Factors:")
print(JuMP.value.(s))
println("\nFinal Demands:")
print(JuMP.value.(f))
println("\n")



Total Flow (Objective value): 16.88571428571429

Scaling Factors:
[34.28571428571429, 3.0, 3.0, 42.857142857142854, 0.8571428571428571]
Final Demands:
[0.0, 0.0, 0.0, 0.0]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -LogLevel 1 -solve -quit (default strategy 1)
Presolve 0 (-10) rows, 0 (-12) columns and 0 (-26) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 16.885714
After Postsolve, objective 16.885714, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 16.88571429 - 0 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



# Appendix A.1.3: Recycling and Down-Cycling Optimization 

In [16]:
model = Model(with_optimizer(Cbc.Optimizer, LogLevel=0))
A=[[1 0 -20 0 30 0];[0 1 -1 0 0 0.1];[0 0 100 -7 0 0];[0 0 0 1 -50 -20]]
B=[[0.25 0.2 2 0 2 10];[0 0 0 0.4 5 2];[0 0 0 0 15 5]]
@variables(model, begin
    f[1:4] ≥ 0
    g[1:3] ≥ 0
    s[1:6] ≥ 0
end)
@constraints(model, begin
    A*s.==f
    B*s.==g
    7*s[4]==300
    constr[i=3:4],f[i]==0
end)

@objective(model, Min, g[1])
JuMP.optimize!(model)

println("\n\nTotal Flow (Objective value): ", JuMP.objective_value(model))
println("\nScaling Factors:")
print(JuMP.value.(s))
println("\nFinal Demands:")
print(JuMP.value.(f))
println("\n");



Total Flow (Objective value): 16.885714285714272

Scaling Factors:
[34.28571428571428, 3.0, 3.0, 42.857142857142854, 0.8571428571428571, 0.0]
Final Demands:
[0.0, 0.0, 0.0, 0.0]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Presolve 0 (-10) rows, 0 (-13) columns and 0 (-31) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 16.885714
After Postsolve, objective 16.885714, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 16.88571429 - 0 iterations time 0.002, Presolve 0.00


## With Mandatory Sorting

In [17]:
model = Model(with_optimizer(Cbc.Optimizer))
A=[[1 0 -20 0 30 0 0];
   [0 1 -1 0 0 0.1 0];
   [0 0 100 -7 0 0 0];
   [0 0 0 1 0 -20 -30];
   [0 0 0 0 -50 0 22]]

B=[[0.25 0.2 2 0 2 10 1];
   [0 0 0 0.4 5 2 0];
   [0 0 0 0 15 5 8]]

@variables(model, begin
    f[1:5] ≥ 0
    g[1:3] ≥ 0
    s[1:7] ≥ 0
end)
@constraints(model, begin
    A*s.==f
    B*s.==g
    7*s[4]==300
    constr[i=3:5],f[i]==0
end)

@objective(model, Min, g[1])
JuMP.optimize!(model)

println("\n\nTotal Flow (Objective value): ", JuMP.objective_value(model))
println("\nScaling Factors:")
print(JuMP.value.(s))
println("\nFinal Demands:")
print(JuMP.value.(f))
println("\n")



Total Flow (Objective value): 19.57142857142857

Scaling Factors:
[41.14285714285714, 3.0, 3.0, 42.857142857142854, 0.6285714285714284, 0.0, 1.4285714285714284]
Final Demands:
[0.0, 0.0, 0.0, 0.0, 0.0]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 0 (-12) rows, 0 (-15) columns and 0 (-37) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 19.571429
After Postsolve, objective 19.571429, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 19.57142857 - 0 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



## Technology Matrix and Intervention Matrix for A.1.3

In [18]:
view(A,1:5,1:7)

5×7 view(::Array{Float64,2}, 1:5, 1:7) with eltype Float64:
 1.0  0.0  -20.0   0.0   30.0    0.0    0.0
 0.0  1.0   -1.0   0.0    0.0    0.1    0.0
 0.0  0.0  100.0  -7.0    0.0    0.0    0.0
 0.0  0.0    0.0   1.0    0.0  -20.0  -30.0
 0.0  0.0    0.0   0.0  -50.0    0.0   22.0

In [19]:
view(B,1:3,1:7)

3×7 view(::Array{Float64,2}, 1:3, 1:7) with eltype Float64:
 0.25  0.2  2.0  0.0   2.0  10.0  1.0
 0.0   0.0  0.0  0.4   5.0   2.0  0.0
 0.0   0.0  0.0  0.0  15.0   5.0  8.0